In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import math as mt
import random as rng
import pickle
from sklearn import preprocessing

Load the data from the last step

In [2]:
data = np.load("grouped.npy", allow_pickle=True)
target = np.load("groupedTarg.npy", allow_pickle=True)

Perform the train and validation split here. The data is split before scaling so that the validation data is scaled after the fitting on the train data- since we are assuming the scenario where a predictive maintenance model will need to handle the scenario of dealing with new data and calling a standardise tranform long after the fitting has taken place. The split is conducted randomly so that 1- this script can be re-run to change the data in the train/test split, 2- any outliers are hopefully spread across both splits

In [3]:
data = data.tolist()
target = target.tolist()

val = []
valTarget = []

valAmount = 30
trainAmount = len(target) - valAmount
print(trainAmount)


for i in range(valAmount):
    index = rng.randint(0, len(target)-1)
    val.append(data.pop(index))
    valTarget.append(target.pop(index))

train = data
trainTarget = target

70


Flatten the data so it can be standardised

In [4]:
def flatten(array):
    flat_array = []
    
    for group in array:
        for i in group:
            flat_array.append(i)
    return flat_array


trainFlat = flatten(train)
trainTargetFlat = flatten(trainTarget)
valFlat = flatten(val)
valTargetFlat = flatten(valTarget)

In [5]:
print(len(trainFlat[0]))
print(len(valFlat))

21
4516


In [6]:
print(trainFlat[0])

[518.67, 642.12, 1584.66, 1401.38, 14.62, 21.61, 553.94, 2388.08, 9057.06, 1.3, 47.49, 522.23, 2388.05, 8130.88, 8.4137, 0.03, 392.0, 2388.0, 100.0, 39.0, 23.375]


Create the standardiser and the PCA object sot he tranformation can be done. Fit the preprocessors to the training data then perform a tranform on the train and validation data

In [7]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()

scaledTrain = scaler.fit_transform(np.array(trainFlat))
scaledVal = scaler.transform(np.array(valFlat))

In [8]:
#PCA

from sklearn.decomposition import PCA
pca = PCA(.96)
scaledTrain = pca.fit_transform(scaledTrain)

scaledVal = pca.transform(scaledVal)

In [9]:
print(len(scaledVal[0]))

11


Save the standardiser and pca so it can tranform other data at a later while using the same parameters 

In [10]:
pickle.dump( scaler, open( "../savedStates/scaler.p", "wb" ) )
pickle.dump(pca, open("../savedStates/pca.p", "wb"))


Regroup the data before saving it again. The data is now ready for the neural network notebook

In [11]:
def regroup(data, target, amount):
    grouped = []
    groupedTarg = []
    
    for i in range(amount):
        grouped.append([])
        groupedTarg.append([])
    
    total = 0
    for i in range(len(grouped)):
        j = total
        if i!= len(grouped)-1:
            while target[j] > target[j+1]:
                grouped[i].append(data[j])
                groupedTarg[i].append(target[j])
                j+=1
            grouped[i].append(data[j])
            groupedTarg[i].append(target[j])
            total = j + 1

    for i in range(total, len(target)):
        grouped[len(grouped)-1].append(data[i])
        groupedTarg[len(grouped)-1].append(target[i])
    
    return grouped, groupedTarg

In [12]:
train, targetTrain = regroup(scaledTrain.tolist(), trainTargetFlat, trainAmount)
val, valTargetTrain = regroup(scaledVal.tolist(), valTargetFlat, valAmount)



np.save("finished/stTrain", np.array(train))
np.save("finished/stTrainTarget", np.array(targetTrain))

np.save("finished/stVal", np.array(val))
np.save("finished/stValTarget", np.array(valTargetTrain))